In [1]:
# Google Play Store Data Analysis
# Internship Project (Extended from Training Project)
# Dataset: Google Play Store Apps Dataset
# Objective:
# - Build interactive dashboards
# - Add new analytics & visualizations
# - Use the SAME dataset used during training

In [2]:
# Task 1st.

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import pytz

In [4]:
df = pd.read_excel(r"C:\Users\DELL\Downloads\google_playstore_apps_data.xlsx")
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_MB,Installs_Clean,Android_Version_Clean,Last_Updated_Clean,Sentiment_Subjectivity,Country
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19.0,10000.0,4.0,2018-01-07,0.56,India
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14.0,500000.0,4.0,2018-01-15,0.97,United States
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8.7,5000000.0,4.0,2018-08-01,0.81,India
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25.0,50000000.0,4.2,2018-06-08,0.72,Brazil
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2.8,100000.0,4.4,2018-06-20,0.41,United Kingdom


In [5]:
df["Last_Updated_Clean"] = pd.to_datetime(df["Last Updated"], errors="coerce")

In [6]:
df["Size_MB"] = df["Size"].replace("Varies with device", np.nan)

df["Size_MB"] = df["Size_MB"].apply(
    lambda x: float(x.replace("M", "")) if isinstance(x, str) and "M" in x
    else float(x.replace("k", "")) / 1024 if isinstance(x, str) and "k" in x
    else np.nan
)

In [7]:
df["Installs_Clean"] = (
    df["Installs"]
    .replace("Free", np.nan)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)

df["Installs_Clean"] = pd.to_numeric(
    df["Installs_Clean"],
    errors="coerce"
)

In [8]:
df = df[
    (df["Rating"] >= 4.0) &
    (df["Size_MB"] >= 10) &
    (df["Last_Updated_Clean"].dt.month == 1)
]

In [9]:
summary = (
    df.groupby("Category")
    .agg(
        Avg_Rating=("Rating", "mean"),
        Total_Reviews=("Reviews", "sum"),
        Total_Installs=("Installs_Clean", "sum")
    )
    .reset_index()
)

In [10]:
top_10 = summary.sort_values(
    by="Total_Installs",
    ascending=False
).head(10)

In [11]:
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

start_time = datetime.strptime("15:00", "%H:%M").time()
end_time = datetime.strptime("17:00", "%H:%M").time()

In [12]:
if start_time <= current_time <= end_time:
    fig = go.Figure()

    fig.add_bar(
        x=top_10["Category"],
        y=top_10["Avg_Rating"],
        name="Average Rating"
    )

    fig.add_bar(
        x=top_10["Category"],
        y=top_10["Total_Reviews"],
        name="Total Reviews"
    )

    fig.update_layout(
        title="Top 10 App Categories by Installs",
        barmode="group",
        xaxis_title="Category",
        yaxis_title="Values"
    )

    fig.show()
else:
    print("Graph available only between 3 PM and 5 PM IST")

Graph available only between 3 PM and 5 PM IST


In [13]:
# Task 2nd.

In [14]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz

In [15]:
df = pd.read_excel(r"C:\Users\DELL\Downloads\google_playstore_apps_data.xlsx")

In [16]:
df['Installs'] = df['Installs'].str.replace('[+,]', '', regex=True)
df = df[df['Installs'].str.isnumeric()]
df['Installs'] = df['Installs'].astype(int)

df = df.dropna(subset=['Category', 'Country'])

In [17]:
df = df[~df['Category'].str.startswith(('A', 'C', 'G', 'S'))]

In [18]:
top_categories = (
    df.groupby('Category')['Installs']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

df = df[df['Category'].isin(top_categories)]

In [19]:
country_installs = (
    df.groupby(['Country', 'Category'])['Installs']
    .sum()
    .reset_index()
)

In [20]:
country_installs['Highlight'] = country_installs['Installs'].apply(
    lambda x: 'Above 1M' if x > 1_000_000 else 'Below 1M'
)

In [21]:
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

show_graph = current_time >= datetime.strptime("18:00", "%H:%M").time() and \
             current_time <= datetime.strptime("20:00", "%H:%M").time()

In [22]:
if show_graph:
    fig = px.choropleth(
        country_installs,
        locations="Country",
        locationmode="country names",
        color="Installs",
        hover_name="Category",
        hover_data=["Highlight"],
        title="Global Installs by Top 5 App Categories",
        color_continuous_scale="Plasma"
    )
    fig.show()

In [23]:
if not show_graph:
    print("⏰ Choropleth Map is visible only between 6 PM and 8 PM IST.")

⏰ Choropleth Map is visible only between 6 PM and 8 PM IST.


In [24]:
# Task 3rd.

In [25]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import pytz

In [26]:
ist = pytz.timezone("Asia/Kolkata")
current_hour = datetime.now(ist).hour

show_graph = 13 <= current_hour < 14

In [27]:
df3 = df.copy()

In [28]:
df3["Installs_Clean"] = (
    df3["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)

df3["Installs_Clean"] = pd.to_numeric(df3["Installs_Clean"], errors="coerce")

In [29]:
df3["Size_MB"] = (
    df3["Size"]
    .astype(str)
    .str.replace("M", "", regex=False)
    .replace("Varies with device", np.nan)
)

df3["Size_MB"] = pd.to_numeric(df3["Size_MB"], errors="coerce")

In [30]:
df3["Android_Ver_Clean"] = (
    df3["Android Ver"]
    .astype(str)
    .str.extract(r"(\d+\.\d+)")
)

df3["Android_Ver_Clean"] = pd.to_numeric(df3["Android_Ver_Clean"], errors="coerce")

In [31]:
df3["Price"] = pd.to_numeric(df3["Price"], errors="coerce").fillna(0)
df3["Revenue"] = df3["Installs_Clean"] * df3["Price"]

In [32]:
df3["Android_Ver_Clean"] = (
    df3["Android Ver"]
    .astype(str)
    .str.extract(r"(\d+\.\d+)")
)

df3["Android_Ver_Clean"] = pd.to_numeric(df3["Android_Ver_Clean"], errors="coerce")

In [33]:
df3 = df3[
    (df3["Installs_Clean"] >= 10000) &
    (df3["Android_Ver_Clean"] > 4.0) &
    (df3["Size_MB"] > 15) &
    (df3["Content Rating"] == "Everyone") &
    (df3["App"].str.len() <= 30)
]

In [34]:
top_3_categories = (
    df3.groupby("Category")["Installs_Clean"]
    .sum()
    .sort_values(ascending=False)
    .head(3)
    .index
)

df3 = df3[df3["Category"].isin(top_3_categories)]

In [35]:
summary = (
    df3.groupby(["Category", "Type"])
    .agg(
        Avg_Installs=("Installs_Clean", "mean"),
        Avg_Revenue=("Revenue", "mean")
    )
    .reset_index()
)

In [36]:
if show_graph:
    fig = go.Figure()

    fig.add_bar(
        x=summary["Category"] + " - " + summary["Type"],
        y=summary["Avg_Installs"],
        name="Average Installs",
        yaxis="y1"
    )

    fig.add_scatter(
        x=summary["Category"] + " - " + summary["Type"],
        y=summary["Avg_Revenue"],
        name="Average Revenue",
        yaxis="y2",
        mode="lines+markers"
    )

    fig.update_layout(
        title="Dual Axis Comparison: Free vs Paid Apps (Top 3 Categories)",
        xaxis_title="Category & App Type",
        yaxis=dict(title="Average Installs"),
        yaxis2=dict(
            title="Average Revenue ($)",
            overlaying="y",
            side="right"
        )
    )

    fig.show()
else:
    print("⏰ Graph visible only between 1 PM IST and 2 PM IST")

⏰ Graph visible only between 1 PM IST and 2 PM IST


In [37]:
#Task 4th.

In [38]:
from datetime import datetime
import pytz

ist = pytz.timezone("Asia/Kolkata")
current_hour = datetime.now(ist).hour

show_graph_4 = (18 <= current_hour < 21)

print("Current IST Hour:", current_hour)
print("Show Graph:", show_graph_4)

Current IST Hour: 11
Show Graph: False


In [39]:
import pandas as pd

df4 = df.copy()

# Reviews clean
df4["Reviews"] = (
    df4["Reviews"].astype(str)
    .str.replace(",", "", regex=False)
)
df4["Reviews"] = pd.to_numeric(df4["Reviews"], errors="coerce")

# Installs clean
df4["Installs"] = (
    df4["Installs"].astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)
df4["Installs"] = pd.to_numeric(df4["Installs"], errors="coerce")

# Date → Month
df4["Last Updated"] = pd.to_datetime(df4["Last Updated"], errors="coerce")
df4["Month"] = df4["Last Updated"].dt.to_period("M").astype(str)

# Category normalize
df4["Category"] = df4["Category"].str.strip().str.upper()

# APPLY ALL CONDITIONS
df4 = df4[
    (~df4["App"].str.lower().str.startswith(("x", "y", "z"))) &
    (~df4["App"].str.contains("s", case=False, na=False)) &
    (df4["Reviews"] > 500) &
    (df4["Category"].str.startswith(("E", "C", "B")))
]

print("Filtered Rows:", df4.shape[0])

Filtered Rows: 0


In [40]:
category_map = {
    "BEAUTY": "सौंदर्य",        # Hindi
    "BUSINESS": "வணிகம்",      # Tamil
    "DATING": "Partnersuche"   # German
}

df4["Category_Label"] = df4["Category"].replace(category_map)

In [41]:
monthly = (
    df4
    .groupby(["Month", "Category_Label"], as_index=False)["Installs"]
    .sum()
    .sort_values("Month")
)

monthly["Growth"] = (
    monthly
    .groupby("Category_Label")["Installs"]
    .pct_change()
)

highlight = monthly[monthly["Growth"] > 0.20]

print("Monthly rows:", monthly.shape[0])
print("High growth rows:", highlight.shape[0])

Monthly rows: 0
High growth rows: 0


In [42]:
import plotly.express as px

if show_graph_4 and not monthly.empty:

    fig = px.line(
        monthly,
        x="Month",
        y="Installs",
        color="Category_Label",
        title="Monthly Install Trends with High Growth Periods"
    )

    # Highlight high growth areas
    for _, row in highlight.iterrows():
        fig.add_vrect(
            x0=row["Month"],
            x1=row["Month"],
            fillcolor="red",
            opacity=0.25,
            line_width=0
        )

    fig.show()

else:
    print("⏰ Graph available only between 6 PM IST and 9 PM IST")

⏰ Graph available only between 6 PM IST and 9 PM IST


In [43]:
# Task 5th.

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import pytz

In [45]:
ist = pytz.timezone("Asia/Kolkata")
current_hour = datetime.now(ist).hour

show_graph_5 = (17 <= current_hour < 19)

print("Current IST Hour:", current_hour)
print("Show 5th Graph:", show_graph_5)

Current IST Hour: 11
Show 5th Graph: False


In [46]:
df = pd.read_excel(r"C:\Users\DELL\Downloads\google_playstore_apps_data.xlsx")

In [47]:
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")

df["Size_MB"] = (
    df["Size"]
    .replace("Varies with device", np.nan)
    .str.replace("M", "")
    .str.replace("k", "")
)

df["Size_MB"] = pd.to_numeric(df["Size_MB"], errors="coerce")

df["Installs"] = (
    df["Installs"]
    .str.replace(",", "")
    .str.replace("+", "")
)

df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

In [48]:
allowed_categories = [
    "Game", "Beauty", "Business", "Comics",
    "Communication", "Dating", "Entertainment",
    "Social", "Events"
]

filtered_df = df[
    (df["Rating"] > 3.5) &
    (df["Reviews"] > 500) &
    (df["Installs"] > 50000) &
    (~df["App"].str.contains("S", case=False, na=False)) &
    (df["Category"].isin(allowed_categories)) &
    (df["Sentiment_Subjectivity"] > 0.5)
]

In [49]:
def translate_category(cat):
    if cat == "Beauty":
        return "सुंदरता"       # Hindi
    elif cat == "Business":
        return "வணிகம்"        # Tamil
    elif cat == "Dating":
        return "Partnersuche"  # German
    else:
        return cat

filtered_df["Category_Display"] = filtered_df["Category"].apply(translate_category)

In [50]:
if show_graph_5:

    plt.figure(figsize=(12, 7))

    for category in filtered_df["Category_Display"].unique():
        cat_data = filtered_df[filtered_df["Category_Display"] == category]

        if category == "Game":
            plt.scatter(
                cat_data["Size_MB"],
                cat_data["Rating"],
                s=cat_data["Installs"] / 1000,
                alpha=0.6,
                label=category,
                color="pink"   # 👈 Highlight Game
            )
        else:
            plt.scatter(
                cat_data["Size_MB"],
                cat_data["Rating"],
                s=cat_data["Installs"] / 1000,
                alpha=0.5,
                label=category
            )

    plt.xlabel("App Size (MB)")
    plt.ylabel("Average Rating")
    plt.title("App Size vs Rating Bubble Chart (Installs as Bubble Size)")
    plt.legend()
    plt.grid(True)
    plt.show()

else:
    print("⏰ Bubble Chart available only between 5 PM – 7 PM IST")

⏰ Bubble Chart available only between 5 PM – 7 PM IST


In [51]:
# Task 6th.

In [52]:
from datetime import datetime
import pytz

ist = pytz.timezone("Asia/Kolkata")
current_hour = datetime.now(ist).hour

show_graph_6 = (16 <= current_hour < 18)

print("Current IST Hour:", current_hour)
print("Show Graph:", show_graph_6)

Current IST Hour: 11
Show Graph: False


In [53]:
df6 = df.copy()

# Clean Reviews
df6["Reviews"] = (
    df6["Reviews"]
    .astype(str)
    .str.replace(",", "", regex=False)
)
df6["Reviews"] = pd.to_numeric(df6["Reviews"], errors="coerce")

# Clean Installs
df6["Installs"] = (
    df6["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)
df6["Installs"] = pd.to_numeric(df6["Installs"], errors="coerce")

# Clean Size (MB)
df6["Size"] = df6["Size"].astype(str).str.replace("M", "", regex=False)
df6["Size_MB"] = pd.to_numeric(df6["Size"], errors="coerce")

# Date → Month
df6["Last Updated"] = pd.to_datetime(df6["Last Updated"], errors="coerce")
df6["Month"] = df6["Last Updated"].dt.to_period("M").astype(str)

In [54]:
df6 = df6[
    (df6["Rating"] >= 4.2) &
    (df6["Reviews"] > 1000) &
    (df6["Size_MB"].between(20, 80)) &
    (~df6["App"].str.contains(r"\d", regex=True, na=False)) &
    (df6["Category"].str.startswith(("T", "P")))
]

df6 = df6.dropna(subset=["Month", "Installs"])

In [55]:
category_translation = {
    "TRAVEL & LOCAL": "Voyage & Local (FR)",
    "PRODUCTIVITY": "Productividad (ES)",
    "PHOTOGRAPHY": "写真 (JP)"
}

df6["Category_Label"] = (
    df6["Category"]
    .str.upper()
    .map(category_translation)
    .fillna(df6["Category"])
)

In [56]:
monthly6 = (
    df6
    .groupby(["Month", "Category_Label"], as_index=False)["Installs"]
    .sum()
    .sort_values("Month")
)

monthly6["Cumulative_Installs"] = (
    monthly6
    .groupby("Category_Label")["Installs"]
    .cumsum()
)

In [57]:
monthly6["Growth"] = (
    monthly6
    .groupby("Category_Label")["Installs"]
    .pct_change()
)

highlight_months = monthly6[monthly6["Growth"] > 0.25]["Month"].unique()

In [58]:
import plotly.express as px

if show_graph_6 and not monthly6.empty:

    fig = px.area(
        monthly6,
        x="Month",
        y="Cumulative_Installs",
        color="Category_Label",
        title="Cumulative Installs Over Time by App Category"
    )

    # Highlight months with >25% growth
    for m in highlight_months:
        fig.add_vrect(
            x0=m,
            x1=m,
            fillcolor="rgba(255,0,0,0.15)",
            line_width=0
        )

    fig.show()

else:
    print("⏰ Graph available only between 4 PM IST and 6 PM IST")

⏰ Graph available only between 4 PM IST and 6 PM IST


In [59]:
# dashbord.

In [61]:
import os
from datetime import datetime
import pytz
import webbrowser

# ---------------- TIME SETUP ----------------
ist = pytz.timezone("Asia/Kolkata")
current_hour = datetime.now(ist).hour

# ---------------- TASK TIME RULES ----------------
tasks = {
    "Task 1": {"time": "3 PM – 5 PM IST", "show": 15 <= current_hour < 17, "graph": "task1.png"},
    "Task 2": {"time": "6 PM – 8 PM IST", "show": 18 <= current_hour < 20, "graph": "task2.png"},
    "Task 3": {"time": "1 PM – 2 PM IST", "show": 13 <= current_hour < 14, "graph": "task3.png"},
    "Task 4": {"time": "6 PM – 9 PM IST", "show": 18 <= current_hour < 21, "graph": "task4.png"},
    "Task 5": {"time": "5 PM – 7 PM IST", "show": 17 <= current_hour < 19, "graph": "task5.png"},
    "Task 6": {"time": "4 PM – 6 PM IST", "show": 16 <= current_hour < 18, "graph": "task6.png"},
}

# ---------------- HTML BLOCKS ----------------
left_active_tasks = ""
right_inactive_tasks = ""

for task, info in tasks.items():
    if info["show"]:
        # ACTIVE TASK → LEFT SIDE (REAL GRAPH)
        left_active_tasks += f"""
        <div class="task-output">
            <h2>{task}</h2>
            <p>🟢 Active Time: {info['time']}</p>
            <img src="graphs/{info['graph']}" class="graph-img.png">
        </div>
        """
    else:
        # INACTIVE TASK → RIGHT SIDE
        right_inactive_tasks += f"""
        <div class="locked-task">
            <h3>{task}</h3>
            <p>⏰ Time: {info['time']}</p>
        </div>
        """

# ---------------- DASHBOARD HTML ----------------
dashboard_html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Task Output Dashboard</title>
    <style>
        body {{
            background:#0f0f0f;
            color:white;
            font-family:Arial;
            margin:0;
        }}
        h1 {{
            text-align:center;
            padding:20px;
        }}
        .container {{
            display:flex;
            padding:20px;
            gap:20px;
        }}
        .left {{
            flex:3;
            display:flex;
            flex-direction:column;
            gap:20px;
        }}
        .right {{
            flex:1;
            display:flex;
            flex-direction:column;
            gap:15px;
        }}
        .task-output {{
            background:#1e1e1e;
            padding:20px;
            border-radius:12px;
        }}
        .graph-img {{
            width:100%;
            margin-top:15px;
            border-radius:10px;
        }}
        .locked-task {{
            background:#2a2a2a;
            padding:15px;
            border-radius:10px;
            opacity:0.6;
            text-align:center;
        }}
        p {{
            color:#aaa;
        }}
    </style>
</head>
<body>

<h1>📊 Task Output Dashboard</h1>
<p style="text-align:center;">Current IST Hour: <b>{current_hour}</b></p>

<div class="container">
    <div class="left">
        {left_active_tasks if left_active_tasks else "<p>No active task right now ⏰</p>"}
    </div>

    <div class="right">
        {right_inactive_tasks}
    </div>
</div>

</body>
</html>
"""

# ---------------- SAVE & OPEN ----------------
os.makedirs("dashboard/graphs", exist_ok=True)
path = "dashboard/dashboard.html"

with open(path, "w", encoding="utf-8") as f:
    f.write(dashboard_html)

webbrowser.open("file://" + os.path.abspath(path))


True